<a href="https://colab.research.google.com/github/AnnaBujniewicz/Masters_thesis/blob/main/4_Headline_Scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# this code  was executed in visual studio code and is posted to colab for clarity's sake
# importing the needed packages
import pandas as pd
import re
import numpy as np
import csv
import newspaper

# definig the function that gets the article titles (using newspaper3k)
def get_titles(url):
    try:
        article = newspaper.Article(url) 
        article.download() 
        article.parse() 
        return article.title
# this prevents the function from throwing an error when a link has been removed
    except:
        return None


# here any of the csv files can be inserted
#FILE NAME HERE
csv_file = 'amzn_data.csv'

# importing the file in a bit of a weird way
# the csv file is space delimeted, but spaces are also present within the tweets, so pandas can't be used
# instead taking first 5 coma separated words as first 5 columns and everything that remains as the 6th one
with open(csv_file, 'r+', encoding='utf-8') as f:
    data = []
    lines = f.readlines()
    for line in lines:
        try:
            text = line.split(' ')
            data.append([text[0], text[1], text[2], text[3], text[4], ' '.join(text[5:])])
        except:
            data.append([None, None, None, None, None, None])
                              
    # regular expression pattern for identifying any url
    regex = r"http\S+"

    # finding all urls within the tweets and appending them to the dataset
    for i, tweet in enumerate(data):
        text = str(tweet[5])
        links = re.findall(regex, text)
        data[i].append(links)

    # creating a dataframe, only selecting the column with the date and a link
    data = pd.DataFrame(data)
    data = data.loc[:,[1,2,6]]
    data.rename(columns={1: 'date', 2: 'time', 6: 'link'},inplace=True)

    # filtering out links after the first day of february
    data["date"] = pd.to_datetime(data["date"], format="%Y/%m/%d", errors = "coerce")
    data = data[data["date"]<"2021-02-01"]

    # in case multiple links were found, putting them in separate rows with the same date
    data = pd.DataFrame({
            col:np.repeat(data[col].values, data["link"].str.len())
            for col in data.columns.drop("link")}
            ).assign(**{"link":np.concatenate(data["link"].values)})[data.columns]
    
    # dropping any duplicate links
    data.drop_duplicates(subset = "link", inplace=True)
    # applying the function that finds article titles to all the links
    data["title"] = data["link"].map(lambda x: get_titles(x))
    # dropping the missing values
    data.dropna(inplace=True)
    #only keeping the titles longer than 5 words
    data = data[data['title'].str.split().str.len().ge(5)]
    #dropping any duplicate headlines
    data.drop_duplicates(subset = "title", inplace=True)
    # saving the file
    data.to_csv("proc_"+csv_file)